In [19]:
import csv
import requests
from bs4 import BeautifulSoup as bs4
import re

#area_name = "神戸"
#area_name = "北阪神・南阪神"
#area_name = "東播磨・北播磨"
#area_name = "中播磨・西播磨"
#area_name = "淡路"
#area_name = "丹波・丹波篠山"
#area_name = "但馬"
area_names = ["神戸", "北阪神・南阪神", "東播磨・北播磨", "中播磨・西播磨", "淡路", "丹波・丹波篠山", "但馬"]

def extract_gotoeat_hyogo_info(area_name):
    csv_filename = "goto_eat_" + area_name + ".csv"
    f = open(csv_filename, mode = 'w', encoding='utf-8', errors = 'ignore')
    writer = csv.writer(f, lineterminator = '\n')

    # CSVヘッダー
    # csv_header = ["店名 ", "住所", "電話", "営業時間", "Webサイト", "定休日"]
    csv_header = ["店名 ", "住所", "電話","Webサイト", "定休日"]
    writer.writerow(csv_header)
    
    area_dic = {"神戸": "1", "北阪神・南阪神": "2", "東播磨・北播磨": "3", "中播磨・西播磨": "4", "淡路": "5", "丹波・丹波篠山": "6", "但馬":"7"}
    if area_name in area_dic:
        area_number = area_dic[area_name]
    else:
        area_number = ""

    for i in range(100):
        page = i + 1
        # html情報取得・解析
        #  url ="https://gotoeat-hyogo.com/search/result"
        url = "https://hyogo-de-tabeyou.com/search"
            
        #param = {"keyword" : search_word, "page" : page} #「伊丹」検索
        param = {"areas[]" : area_number, "page" : page} #「伊丹」検索
        res = requests.get(url, param)
        search_result = bs4( res.text, "html.parser")

        # 店舗名、店舗住所の取得
        store_list = search_result.select("div.search-list")
        if(len(store_list) == 0):
            print("page:", i)
            break

        for store in store_list:
            #店名
            store_name = store.select("div.shop-list-ttl")[0].text
            #住所
            store_address = store.select("div.shop-list-address")[0].text.replace(" ", "").replace("\n", "").replace("住所：", "")
            store_address = re.sub('(〒[0-9]{3}-[0-9]{4})', "", store_address)
            #電話番号
            store_tel_number = store.select("div.shop-list-tel")[0].text.replace(" ", "").replace("\n", "").replace("TEL:", "")

            #詳細ページ解析
            store_detail_url = store.select("a.shop-list-btn")[0]["href"]
            store_res = requests.get(store_detail_url)
            store_detail = bs4( store_res.text, "html.parser")
            store_detail_table = store_detail.select("div.con_in")

            #営業時間
            #store_business_hours_title = store_detail_table[0].find("label", text="営業時間")
            #store_business_hours = store_business_hours_title.next_sibling.next_sibling.text.replace(" ", "").replace("\n", "") if store_business_hours_title else ""
            #print(store_business_hours)
            #print("------------------------------")
            #Webサイト
            store_web_title = store_detail_table[0].find("label", text="店舗サイト")
            store_web = store_web_title.next_sibling.next_sibling.text.replace(" ", "").replace("\n", "") if store_web_title else ""
            #定休日
            store_regular_holiday_title = store_detail_table[0].find("label", text="定休日")
            store_memo = "定休日:" + store_regular_holiday_title.next_sibling.next_sibling.text.replace(" ", "").replace("\n", "") if store_regular_holiday_title else ""

            csv_list = []
            csv_list.append(store_name)
            csv_list.append(store_address)
            csv_list.append(store_tel_number)
            #csv_list.append(store_business_hours)
            csv_list.append(store_web)
            csv_list.append(store_memo)
            writer.writerow(csv_list)

    f.close()

for area_name in area_names:
    extract_gotoeat_hyogo_info(area_name)


page: 85
page: 66
page: 42
page: 31
page: 11
page: 4
page: 11
